In [52]:
from statistics import mean
import torch
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer import Importance, EmpiricalMarginal
import matplotlib.pyplot as plt
%matplotlib inline

pyro.set_rng_seed(101)

In [53]:
def model():
    A_alias = ['off', 'on']
    B_alias = ['off', 'on']
    C_alias = ['off', 'on']

    A_prob = torch.tensor([0.5,0.5])
    A = pyro.sample("A", dist.Categorical(probs = A_prob))
    B_prob = torch.tensor([[0.8,0.2],[0.1,0.9]])
    B = pyro.sample("B", dist.Categorical(probs = B_prob[A]))
    C_prob = torch.tensor([[[0.9,0.1],[0.99,0.01]], [[0.1,0.9],[0.4,0.6]]])
    C = pyro.sample("C", dist.Categorical(probs = C_prob[B][A]))
    return{'A': A, 'B': B, 'C': C}

In [54]:
#P(A = on | B = on, C = on)
conditioned_model = pyro.condition(model, data = {'B':torch.tensor(1),'C':torch.tensor(1)})
A_posterior = pyro.infer.Importance(conditioned_model, num_samples=1000).run()
A_marginal = EmpiricalMarginal(A_posterior,"A")
A_samples = [A_marginal().item() for _ in range(1000)]
A_unique, A_counts = np.unique(A_samples, return_counts=True)
A_counts[1]/sum(A_counts)

0.732

In [55]:
#P(A = on | do(B = on), C = on)
b_on = pyro.do(model, data={'B': torch.tensor(1)})
b_on_conditioned = pyro.condition(b_on, data={'C': torch.tensor(1)})
A_posterior = pyro.infer.Importance(b_on_conditioned, num_samples=1000).run()
A_marginal = EmpiricalMarginal(A_posterior,"A")
A_samples = [A_marginal().item() for _ in range(1000)]
A_unique, A_counts = np.unique(A_samples, return_counts=True)
A_counts[1]/sum(A_counts)

0.407